# Downloading data about the channel & videos
## Analysis of Unstructured Data
### Anna Szymanek 230042

In [1]:
from googleapiclient.discovery import build
import pandas as pd

In [2]:
channel_name = 'Eva zu Beck'
csv_file_name = 'EvazuBeck_data.csv'

In [3]:
youTubeApiKey = 'myApiKey'
youtube = build('youtube', 'v3', developerKey = youTubeApiKey)

In [4]:
snippets = youtube.search().list(part = 'snippet', type = 'channel', q = channel_name).execute()

In [5]:
channelId = snippets['items'][0]['snippet']['channelId']
print(channel_name, 'channel id is', channelId)

Eva zu Beck channel id is UCDTINI9skkeZNY2ZXnBqIkQ


In [6]:
stats = youtube.channels().list(part='statistics', id = channelId).execute()


In [7]:
stats['items']

[{'kind': 'youtube#channel',
  'etag': 'L86jpcDy63XHhwXKR3YdYTnOkG8',
  'id': 'UCDTINI9skkeZNY2ZXnBqIkQ',
  'statistics': {'viewCount': '45725046',
   'subscriberCount': '854000',
   'hiddenSubscriberCount': False,
   'videoCount': '114'}}]

In [8]:
content = youtube.channels().list(id = channelId, part = 'contentDetails').execute()

In [9]:
content['items']

[{'kind': 'youtube#channel',
  'etag': 'GgtKBzllpA5hQ2pD58Vdk8vC5KU',
  'id': 'UCDTINI9skkeZNY2ZXnBqIkQ',
  'contentDetails': {'relatedPlaylists': {'likes': '',
    'favorites': '',
    'uploads': 'UUDTINI9skkeZNY2ZXnBqIkQ'}}}]

In [10]:
UploadId = content ['items'][0]['contentDetails']['relatedPlaylists']['uploads']

In [11]:
allVideos = []
nextPage_token = None
res = youtube.playlistItems().list(playlistId = UploadId, maxResults = 50, part = 'snippet', pageToken = nextPage_token).execute()

In [12]:
while 1:
    res = youtube.playlistItems().list(playlistId = UploadId, maxResults = 50, part = 'snippet', pageToken = nextPage_token).execute()
    allVideos += res['items']
    nextPage_token = res.get('nextPageToken')
    if nextPage_token is None:
        break

In [15]:
video_ids = list(map(lambda x: x['snippet']['resourceId']['videoId'], allVideos))
print(len(video_ids))

114


In [16]:
stats = []
for i in range (0,len(video_ids),40):
    res = youtube.videos().list(id = ','.join(video_ids[i:i+40]), part = 'statistics').execute()
    stats += res['items']

114


In [17]:
titleVD = []
liked = []
disliked = []
views = []
comment = []
videoid = []
published_date = []
video_description = []


In [18]:
for i in range(0,len(video_ids)):
    videoid.append((allVideos[i])['snippet']['resourceId']['videoId'])
    titleVD.append((allVideos[i])['snippet']['title'])
    published_date.append((allVideos[i])['snippet']['publishedAt'])
    video_description.append((allVideos[i])['snippet']['description'])
    liked.append(int((stats[i])['statistics']['likeCount']))
    disliked.append(int((stats[i])['statistics']['dislikeCount']))
    views.append(int((stats[i])['statistics']['viewCount']))
    comment.append(int((stats[i])['statistics']['commentCount']))

In [20]:
df = pd.DataFrame({'VideoID':videoid, 'Title':titleVD, 'VideoDescription':video_description, 'PublishedDate':published_date,  'Likes':liked, 'Disliked':disliked, 'Views':views, 'Comment':comment})
df.head()

,VideoID,Title,VideoDescription,PublishedDate,Likes,Disliked,Views,Comment
0,7KobMsd7jLw,Running a Marathon in Yemen (& BIG 2021 News),→ Join me for daily adventures on Instagram: ...,2021-01-16T08:54:47Z,5784,96,100029,844
1,MYDsjJivU8s,BEAUTIFUL Pakistani Wedding in the Mountains (...,→ Join me for daily adventures on Instagram: ...,2021-01-03T11:32:14Z,32350,929,1146101,4536
2,gSKnFpQwKBA,I Became a Traditional Bride in Kosovo for 1 Day,→ Join me for daily adventures on Instagram: ...,2020-12-27T08:29:44Z,6385,103,148132,1091
3,MPb_Ybh9uAo,Inside an Albanian Village Guesthouse,→ Join me for daily adventures on Instagram: ...,2020-12-20T15:29:38Z,12921,438,577260,2011
4,R4xfHIZ6TQM,Trekking 2 Days Alone in ALBANIA,→ Join me for daily adventures on Instagram: ...,2020-12-13T13:22:05Z,13654,149,306423,1948


In [22]:
df.to_csv(csv_file_name, index = False)